In [1]:
# Imports

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from numpy.matlib import repmat

from timeit import default_timer as timer

import tqdm

import scipy

from handythread import foreach
import cython

#import warnings
#warnings.filterwarnings("ignore")

In [3]:
%%cython

a: cython.int = 0
for i in range(10):
    a += i
print(a)

45


In [2]:
%load_ext cython

In [36]:
%%cython

import cython

import numpy as np
cimport numpy as np

# differential evolution search of the two-dimensional sphere objective function
from numpy.random import rand
from numpy.random import choice
from numpy import asarray
from numpy import clip
from numpy import argmin
from numpy import min
from numpy import around

import scipy
import scipy.io as spio

import tqdm

def loadmat(filename):
    '''
    this function should be called instead of direct spio.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    '''
    def _check_keys(d):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in d:
            if isinstance(d[key], spio.matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
        return d

    def _has_struct(elem):
        """Determine if elem is an array and if any array item is a struct"""
        return isinstance(elem, np.ndarray) and any(isinstance(
                    e, scipy.io.matlab.mio5_params.mat_struct) for e in elem)

    def _todict(matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, spio.matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif _has_struct(elem):
                d[strg] = _tolist(elem)
            else:
                d[strg] = elem
        return d

    def _tolist(ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, spio.matlab.mio5_params.mat_struct):
                elem_list.append(_todict(sub_elem))
            elif _has_struct(sub_elem):
                elem_list.append(_tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list
    data = scipy.io.loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_keys(data)


# Matlab code conversion

# Read matlab file instead of excelfile (easier conversion)
data = loadmat('matlab.mat')

# Definir o numero de resources usados

numGen = np.arange(1, data['data']['parameterData']['resources']['numGen'] + 1)
numLoad = np.arange(1, data['data']['parameterData']['resources']['numLoad'] + 1)
numStor = np.arange(1, data['data']['parameterData']['resources']['numStor'] + 1)
numV2G = np.arange(1, data['data']['parameterData']['resources']['numV2G'] + 1)
numCStat = np.arange(1, data['data']['parameterData']['resources']['numCStat'] + 1)
numPeriod = np.arange(1, data['data']['parameterData']['resources']['period'] + 1)
numBus = np.array([1]) #np.arange(1) # forced to 1
nOwner = np.arange(1, data['data']['parameterData']['resources']['owners'] + 1)

# Define as strings com os recursos para mandar para GAMS

gen = np.max(numGen)
Load = np.max(numLoad)
stor = np.max(numStor)
v2g = np.max(numV2G)
cs = np.max(numCStat)
period = np.max(numPeriod)
Bus = np.max(numBus)

#% Definir as matrizes com os dados para o GAMS

# General Info
pMaxImp = data['data']['parameterData']['generalInfo']['P_Max_Imp']
pMaxExp = data['data']['parameterData']['generalInfo']['P_Max_Exp']
buyPrice = data['data']['parameterData']['generalInfo']['Energy_Buy_Price']
sellPrice = data['data']['parameterData']['generalInfo']['Energy_Sell_Price']

# Geradores
genLimit = data['data']['generator']['limit'][numGen[0]-1:numGen[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
genInfo = data['data']['generator']['info']

# Cargas
loadLimit = data['data']['load']['limit'][numLoad[0]-1:numLoad[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]

# Baterias
storLimit = data['data']['storage']['limit'][numStor[0]-1:numStor[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
storInfo = data['data']['storage']['info']

# Veiculos
v2gLimit = data['data']['vehicle']['limit']
v2gInfo = data['data']['vehicle']['info']

# Charging Station
csLimit = data['data']['cstation']['limit'][numCStat[0]-1:numCStat[-1]+1, numPeriod[0]-1:numPeriod[-1]+1, :]
csInfo = data['data']['cstation']['info']

# Connect EV to Charging Station
EV_CS_Info = data['data']['vehicle']['timeInfo']['V2GinCS']

# * Define the several sets structure used in the gams optimization process, 
# these sets depends the number of resources used in matlab code

_t = np.arange(period)
_gen = np.arange(gen)
_Load = np.arange(Load)
_stor = np.arange(stor)
_v2g = np.arange(v2g)
_cs = np.arange(cs)

#positive variables

genActPower = np.ones(shape=(len(_gen), len(_t)))
genExcActPower = np.ones(shape=(len(_gen), len(_t)))
pImp = np.ones(shape=(len(_t)))
pExp = np.ones(shape=(len(_t)))

# Load variables
loadRedActPower = np.ones(shape=(len(_Load), len(_t)))
loadCutActPower = np.ones(shape=(len(_Load), len(_t)))
loadENS = np.ones(shape=(len(_Load), len(_t)))

# Storage variables
storDchActPower = np.ones(shape=(len(_stor), len(_t)))
storChActPower = np.ones(shape=(len(_stor), len(_t)))
storEminRelaxStor = np.ones(shape=(len(_stor), len(_t)))
storEnerState = np.ones(shape=(len(_stor), len(_t)))

# V2G variables
v2gDchActPower = np.ones(shape=(len(_v2g), len(_t)))
v2gChActPower = np.ones(shape=(len(_v2g), len(_t)))
v2gEnerState = np.ones(shape=(len(_v2g), len(_t)))
v2gEminRelaxEv = np.ones(shape=(len(_v2g), len(_t)))

# Charging Stations
csActPower = np.ones(shape=(len(_cs), len(_t)))
csActPowerNet = np.ones(shape=(len(_cs), len(_t)))

# # binary variables

genXo = np.zeros(shape=(len(_gen), len(_t)))
loadXo = np.zeros(shape=(len(_Load), len(_t)))

storChXo = np.zeros(shape=(len(_stor), len(_t)))
storDchXo = np.zeros(shape=(len(_stor), len(_t)))

v2gChXo = np.zeros(shape=(len(_v2g), len(_t)))
v2gDchXo = np.zeros(shape=(len(_v2g), len(_t)))



# Define dictionaries
dict_var = {'genActPower': genActPower,
            'genExcActPower': genExcActPower,
            'pImp': pImp,
            'pExp': pExp,
            'loadRedActPower': loadRedActPower,
            'loadCutActPower': loadCutActPower,
            'loadENS': loadENS,
            'storDchActPower': storDchActPower,
            'storChActPower': storChActPower,
            'EminRelaxStor': storEminRelaxStor,
            'storEnerState': storEnerState,
            'v2gDchActPower': v2gDchActPower,
            'v2gChActPower': v2gChActPower,
            'v2gEnerState': v2gEnerState,
            'EminRelaxEv': v2gEminRelaxEv,
            'csActPower': csActPower,
            'csActPowerNet': csActPowerNet,
            'genXo': genXo,
            'loadXo': loadXo,
            'storChXo': storChXo,
            'storDchXo': storDchXo,
            'v2gDchXo': v2gDchXo,
            'v2gChXo': v2gChXo}

dict_par = {'genLimit': genLimit,
            'genInfo': genInfo,
            'loadLimit': loadLimit,
            'loadActPower': loadLimit[:, :, 0],
            'storLimit': storLimit,
            'storInfo': storInfo[:, :12],
            'v2gLimit': v2gLimit,
            'v2gInfo': v2gInfo,
            'csLimit': csLimit,
            'csInfo': csInfo,
            'EV_CS_Info': EV_CS_Info,
            'buyPrice': buyPrice,
            'sellPrice': sellPrice,
            't': _t,
            'gen': _gen,
            'load': _Load,
            'stor': _stor,
            'v2g': _v2g}

minVals = {'genActPower': np.array(np.zeros((1, genActPower.ravel().shape[0]))),
           'genExcActPower': np.array(np.zeros((1, genExcActPower.ravel().shape[0]))),
           'pImp': np.array(np.zeros((1, pImp.ravel().shape[0]))),
           'pExp': np.array(np.zeros((1, pExp.ravel().shape[0]))),
           'loadRedActPower': np.array(np.zeros((1, loadRedActPower.ravel().shape[0]))),
           'loadCutActPower': np.array(np.zeros((1, loadCutActPower.ravel().shape[0]))),
           'loadENS': np.array(np.zeros((1, loadENS.ravel().shape[0]))),
           'storDchActPower': np.array(np.zeros((1, storDchActPower.ravel().shape[0]))),
           'storChActPower': np.array(np.zeros((1, storChActPower.ravel().shape[0]))),
           'EminRelaxStor': np.array(np.zeros((1, storEminRelaxStor.ravel().shape[0]))),
           'storEnerState': np.array(np.zeros((1, storEnerState.ravel().shape[0]))),
           'v2gDchActPower': np.array(np.zeros((1, v2gDchActPower.ravel().shape[0]))),
           'v2gChActPower': np.array(np.zeros((1, v2gChActPower.ravel().shape[0]))),
           'v2gEnerState': np.array(np.zeros((1, v2gEnerState.ravel().shape[0]))),
           'EminRelaxEv': np.array(np.zeros((1, v2gEminRelaxEv.ravel().shape[0]))),
           'csActPower': (csInfo[:, 5] * -1 * csActPower.transpose()).transpose().ravel(),
           'csActPowerNet': np.array(np.zeros((1, csActPowerNet.ravel().shape[0]))),
           'genXo': np.array(np.zeros((1, genXo.ravel().shape[0]))),
           'loadXo': np.array(np.zeros((1, loadXo.ravel().shape[0]))),
           'storChXo': np.array(np.zeros((1, storChXo.ravel().shape[0]))),
           'storDchXo': np.array(np.zeros((1, storDchXo.ravel().shape[0]))),
           'v2gDchXo': np.array(np.zeros((1, v2gDchXo.ravel().shape[0]))),
           'v2gChXo': np.array(np.zeros((1, v2gChXo.ravel().shape[0])))}


maxVals = {'genActPower': 200*np.array(np.ones((1, genActPower.ravel().shape[0]))),
           'genExcActPower': 200*np.array(np.ones((1, genExcActPower.ravel().shape[0]))),
           'pImp': pMaxImp.ravel(),
           'pExp': pMaxExp.ravel(),
           #'loadRedActPower': 200*np.array(np.ones((1, loadRedActPower.ravel().shape[0]))),
           #par['loadLimit'][l, t, 2]
           'loadRedActPower': loadLimit[:, :, 2].ravel(),
           #'loadCutActPower': 200*np.array(np.ones((1, loadCutActPower.ravel().shape[0]))),
           'loadCutActPower': loadLimit[:, :, 2].ravel(),
           'loadENS': 200*np.array(np.ones((1, loadENS.ravel().shape[0]))),
           'storDchActPower': 200*np.array(np.ones((1, storDchActPower.ravel().shape[0]))),
           'storChActPower': 200*np.array(np.ones((1, storChActPower.ravel().shape[0]))),
           'EminRelaxStor': 200*np.array(np.ones((1, storEminRelaxStor.ravel().shape[0]))),
           'storEnerState': np.array(np.ones((1, storEnerState.ravel().shape[0]))),
           'storEnerState': (storInfo[:, 5] * storEminRelaxStor.transpose()).transpose().ravel(),
           'v2gDchActPower': 200*np.array(np.ones((1, v2gDchActPower.ravel().shape[0]))),
           'v2gChActPower': 200*np.array(np.ones((1, v2gChActPower.ravel().shape[0]))),
           'v2gEnerState': (v2gInfo[:, 4] * v2gEnerState.transpose()).transpose().ravel(),
           'EminRelaxEv': 200*np.array(np.ones((1, v2gEminRelaxEv.ravel().shape[0]))),
           'csActPower': (csInfo[:, 4] * csActPower.transpose()).transpose().ravel(),
           'csActPowerNet': 200*np.array(np.ones((1, csActPowerNet.ravel().shape[0]))),
           'genXo': np.array(np.ones((1, genXo.ravel().shape[0]))),
           'loadXo': np.array(np.ones((1, loadXo.ravel().shape[0]))),
           'storChXo': np.array(np.ones((1, storChXo.ravel().shape[0]))),
           'storDchXo': np.array(np.ones((1, storDchXo.ravel().shape[0]))),
           'v2gDchXo': np.array(np.ones((1, v2gDchXo.ravel().shape[0]))),
           'v2gChXo': np.array(np.ones((1, v2gChXo.ravel().shape[0])))}


# Turn and recover vector for encoding
def encode(var_dictionary):
    result_concat = np.concatenate([var_dictionary[v].ravel() for v in var_dictionary.keys()])
    return result_concat

def decode(var_vector, var_dictionary):
    result_decoded = {}
    var_idx = [var_dictionary[v].ravel().shape[0] for v in var_dictionary.keys()]

    current_index = 0
    dictionary_names = list(var_dictionary.keys())
    
    for idx in np.arange(len(dictionary_names)):
        result_index = current_index + var_idx[idx]
        result_decoded[dictionary_names[idx]] = np.reshape(var_vector[current_index:result_index], 
                                                           var_dictionary[dictionary_names[idx]].shape)
        
        current_index = result_index
        
    return result_decoded



# Generation checks
def check_pop_gens(val, par, val_dictionary):
    # Decode the solution vector
    pop = decode(val, val_dictionary)
    
    # Bound the boolean variables
    pop['genXo'] = (pop['genXo'] > 0.5).astype(int)
    
    g: cython.int
        
    # Go through generators
    for g in range(pop['genActPower'].shape[0]):
        
        # Check generator type
        if par['genInfo'][g, 4] != 1:
            
            # Maximum generation in generators with normal contract
            pop['genActPower'][g, :][pop['genActPower'][g, :] > par['genLimit'][g, :, 0]] = par['genLimit'][g, :, 0][pop['genActPower'][g, :] > par['genLimit'][g, :, 0]]

            # Minimum generation
            pop['genActPower'][g, :][pop['genActPower'][g, :] < par['genInfo'][g, 6] * pop['genXo'][g, :]] = (par['genInfo'][g, 6] * pop['genXo'][g, :])[pop['genActPower'][g, :] < par['genInfo'][g, 6] * pop['genXo'][g, :]]
                
        elif par['genInfo'][g, 4] != 2:
            # Generate the genExcActPower variable according to the equality equation
            pop['genExcActPower'][g, :] = par['genLimit'][g, :, 0] - pop['genActPower'][g, :]
            
    return encode(pop)

# Check loads
def check_pop_loads(val, par, val_dictionary):
    # Decode the solution vector
    pop = decode(val, val_dictionary)
    
    # Bound the boolean variables
    pop['loadXo'] = (pop['loadXo'] > 0.5).astype(int)
    
    l: cython.int
    
    # Go through the loads
    pop['loadCutActPower'][:, :] = par['loadLimit'][:, :, 3] * pop['loadXo'][:, :]
    
    temp_val = par['loadActPower'] - pop['loadRedActPower'] - pop['loadCutActPower']
    pop['loadENS'][pop['loadENS'] > temp_val] = temp_val[pop['loadENS'] > temp_val]
    
    return encode(pop)

# Check storage
def check_pop_stor(val, par, val_dictionary):
    
    # Decode the solution vector
    pop = decode(val, val_dictionary)
    
    # Bound the binary variables
    pop['storChXo'] = (pop['storChXo'] > 0.5).astype(int)
    pop['storDchXo'] = (pop['storDchXo'] > 0.5).astype(int)
    
    s: cython.int
    t: cython.int
    mask = None
    
    for s in range(pop['storDchActPower'].shape[0]):
        # Timestep
        for t in np.arange(1, pop['storDchActPower'].shape[1]):
            # Storage charge and discharge incompatibility
            if pop['storChXo'][s, t] + pop['storDchXo'][s, t] > 1:
                pop['storChXo'][s, t] = 1
                pop['storDchXo'][s, t] = 0
                
    # Discharge inequality
    mask = pop['storDchActPower'] > par['storLimit'][:, :, 1] * pop['storDchXo']
    pop['storDchActPower'][mask] = (par['storLimit'][:, :, 1] * pop['storDchXo'])[mask]

    # Storage charge check
    mask = pop['storChActPower'] > par['storLimit'][:, :, 0] * pop['storChXo']
    pop['storChActPower'][mask] = (par['storLimit'][:, :, 0] * pop['storChXo'])[mask]
                
    # Storage relax constraint
    temp_val = [par['storInfo'][:, 6] * par['storInfo'][:, 5] - pop['EminRelaxStor'][:, t] for t in range(par['t'].shape[0])]
    temp_val = np.array(temp_val).transpose()
    mask = pop['storEnerState'] < temp_val
    pop['storEnerState'][mask] = temp_val[mask]
                
    # Battery update rules - t=0
    pop['storEnerState'][:, 0] = par['storInfo'][:, 5] * par['storInfo'][:, 9]/100 + pop['storChActPower'][:, 0] * par['storInfo'][:, 7]/100 - pop['storDchActPower'][:, 0]/par['storInfo'][:, 8] / 100
        
    # Battery update rules - t > 0
    temp_val = [pop['storChActPower'][:, t] * par['storInfo'][:, 7]/100 - pop['storDchActPower'][:, t]/par['storInfo'][:, 8]/100 for t in range(1, par['t'].shape[0])]
    temp_val = np.array(temp_val).transpose()
    pop['storEnerState'][:, 1:] = pop['storEnerState'][:, :-1] + temp_val

    return encode(pop)

# Check v2g
def check_pop_v2g(val, par, val_dictionary):
    
    # Decode the solution vector
    pop = decode(val, val_dictionary)
    
    # Bound binary variables
    pop['v2gChXo'] = (pop['v2gChXo'] > 0.5).astype(int)
    pop['v2gDchXo'] = (pop['v2gDchXo'] > 0.5).astype(int)
    
    mask = None
    
    v: cython.int
    t: cython.int
    
    # V2G constraints
    for v in range(pop['v2gChActPower'].shape[0]):
        
        # Timestep
        for t in np.arange(pop['v2gChActPower'].shape[1]):
            
            # Disable charge and discharge in the same period
            if pop['v2gChXo'][v, t] + pop['v2gDchXo'][v, t] > 1:
                pop['v2gChXo'][v, t] = 1
                pop['v2gDchXo'][v, t] = 0
                
    # Discharge rate
    temp_val = par['v2gLimit'][:, :, 4] * par['v2gLimit'][:, :, 0] * pop['v2gDchXo']
    mask = pop['v2gDchActPower'] > temp_val
    pop['v2gDchActPower'][mask] = temp_val[mask]
    
    
    # Charge rate
    temp_val = par['v2gLimit'][:, :, 3] * par['v2gLimit'][:, :, 0] * pop['v2gChXo']
    mask = pop['v2gChActPower'] > temp_val
    pop['v2gChActPower'][mask] = temp_val[mask]
             
        
    # Energy Limits Eq1
    mask = pop['v2gEnerState'] < par['v2gLimit'][:, :, 2] - pop['EminRelaxEv']
    mask2 = par['v2gLimit'][:, :, 0] == 1
    pop['v2gEnerState'][mask & mask2] = par['v2gLimit'][:, :, 2][mask & mask2] - pop['EminRelaxEv'][mask & mask2]
    

    # Energy Limits Eq2
    mask = [(par['v2gLimit'][:, t, 0] == 0) & (par['v2gLimit'][:, t+1, 0] == 0) & (par['v2gLimit'][:, t, 2] == 0)
            for t in range(par['t'].shape[0] - 2)]
    
    mask2 = [pop['v2gEnerState'][:, t] < par['v2gInfo'][:, 4] - pop['EminRelaxEv'][:, t] 
             for t in range(par['t'].shape[0] - 2)]
    
    mask3 = np.array(mask) & np.array(mask2)
    mask3 = mask3.transpose()
    
    temp_val = [par['v2gInfo'][:, 4] - pop['EminRelaxEv'][:, t] for t in range(par['t'].shape[0])]
    temp_val = np.array(temp_val).transpose()
    
    pop['v2gEnerState'][:, :-2][mask3] = temp_val[:, :-2][mask3]

            
    # Energy Limits Eq3
    mask = par['v2gLimit'][:, -1, 0] == 0
    mask2 = par['v2gLimit'][:,-1, 2] == 0
    
    temp_val = [par['v2gInfo'][:, 4] - pop['EminRelaxEv'][:, t]
                for t in range(par['t'].shape[0])]
    temp_val = np.array(temp_val).transpose()
    
    pop['v2gEnerState'][:, -1][mask & mask2] = temp_val[:, -1][mask & mask2]
    
    
    # Energy Balance Initial Eq
    mask = par['v2gLimit'][:, 0, 0] == 0
    temp_val = par['v2gLimit'][:, 0, 1] + pop['v2gChActPower'][:, 0] * par['v2gInfo'][:, 7] - pop['v2gDchActPower'][:, 0] / par['v2gInfo'][:, 8]
    pop['v2gEnerState'][:, 0][mask] = temp_val[mask]
    

    # Energy Balance 1
    mask = par['v2gLimit'][:, :-1, 0] == 1
    mask2 = par['v2gLimit'][:, 1:, 0] == 1
    
    temp_val = [pop['v2gEnerState'][:, t-1] + par['v2gLimit'][:, t, 1] + pop['v2gChActPower'][:, t] * par['v2gInfo'][:, 7] - pop['v2gDchActPower'][:, t] / par['v2gInfo'][:, 8]
                for t in range(1, par['t'].shape[0])]
    temp_val = np.array(temp_val).transpose()
    pop['v2gEnerState'][:, 1:][mask & mask2] = temp_val[mask & mask2]
    

    # Energy Balance 2
    mask = par['v2gLimit'][:, :-1, 0] == 0
    #mask2 = par['v2gLimit'][:, 1:, 0] == 1 - same as previous mask
    temp_val = [par['v2gLimit'][:, t, 1] + pop['v2gChActPower'][:, t] * par['v2gInfo'][:, 8] + pop['v2gDchActPower'][:, t] / par['v2gInfo'][:, 9]
                for t in range(1, par['t'].shape[0])] 
    temp_val = np.array(temp_val).transpose()
    pop['v2gEnerState'][:, 1:][mask & mask2] = temp_val[mask & mask2]
    
    return encode(pop)

# Check charging stations
def check_pop_cs(val, par, val_dictionary):
    # Decode the solution vector
    pop = decode(val, val_dictionary)
    
    c: cython.int
    
    # Charging station constraints
    for c in range(pop['csActPower'].shape[0]):
        
        # Timesteps
        for t in range(pop['csActPower'].shape[1]):
            
            temp_val = 0
            temp_val2 = 0
            for v in np.arange(par['v2g'][-1]):
                if par['EV_CS_Info'][v, c, t] > 0:
                    temp_val += (pop['v2gChActPower'][v, t]  - pop['v2gDchActPower'][v, t])
                    temp_val2 += (pop['v2gChActPower'][v, t] / (par['csInfo'][c, 6] / 100) - pop['v2gDchActPower'][v, t] * par['csInfo'][c, 7] / 100) 
                    
            pop['csActPower'][c, t] = temp_val
            pop['csActPowerNet'][c, t] = temp_val2
    
    return encode(pop)

# define mutation operation
def mutation(x, F):
    return x[0] + F * (x[1] - x[2])


# define boundary check operation
def check_bounds(mutated, bounds):
    mutated_bound = [clip(mutated[i], bounds[i, 0], bounds[i, 1]) for i in range(len(bounds))]
    return mutated_bound

def check_pops(mutated):
    mutated = check_pop_gens(mutated, dict_par, dict_var)
    mutated = check_pop_loads(mutated, dict_par, dict_var)
    mutated = check_pop_stor(mutated, dict_par, dict_var)
    mutated = check_pop_v2g(mutated, dict_par, dict_var)
    mutated = check_pop_cs(mutated, dict_par, dict_var)
    
    return mutated

# define crossover operation
def crossover(mutated, target, dims, cr):
    # generate a uniform random value for every dimension
    p = rand(dims)
    # generate trial vector by binomial crossover
    trial = [mutated[i] if p[i] < cr else target[i] for i in range(dims)]
    return trial


def eso_obj(vals):
    par = dict_par

    #obj_value = []
    var = decode(vals, dict_var)
    
    temp_gens: cython.float[24] = sum(var['genActPower'] * par['genLimit'][:, :, 2] + var['genExcActPower'] * par['genLimit'][:, :, 4])
    sum_gens: cython.float = sum(temp_gens)
    
    temp_loads: cython.float[24] = sum(var['loadRedActPower'] * par['loadLimit'][:, :, 6] + var['loadCutActPower'] * par['loadLimit'][:, :, 7] + var['loadENS'] * par['loadLimit'][:, :, 9])
    sum_loads:cython.float = sum(temp_loads)
    
    temp_stor: cython.float[24] = sum(var['storDchActPower'] * par['storLimit'][:, :, 3] - var['storChActPower'] * par['storLimit'][:, :, 2] + var['EminRelaxStor'] * 200 )
    sum_stor: cython.float = sum(temp_stor)
    
    temp_v2g: cython.float[24] = sum(var['v2gDchActPower'] * par['v2gLimit'][:, :, 6] - var['v2gChActPower'] * par['v2gLimit'][:, :, 5] + var['EminRelaxEv'] * 200)
    sum_v2g: cython.float = sum(temp_v2g)
    
    temp_rest: cython.float = sum(var['pImp'] * par['buyPrice'] - var['pExp'] * par['sellPrice'])
    sum_rest: cython.float = temp_rest


    # Objective function values
    #temp_gens = sum([var['genActPower'][g, t] * par['genLimit'][g, t, 2] + var['genExcActPower'][g, t] * par['genLimit'][g, t, 4]
    #                 for t in np.arange(par['t'][-1]) for g in np.arange(par['gen'][-1])])

    #temp_loads = sum([var['loadRedActPower'][l, t] * par['loadLimit'][l, t, 6] + var['loadCutActPower'][l, t] * par['loadLimit'][l, t, 7] + var['loadENS'][l, t] * par['loadLimit'][l, t, 9] 
    #                  for t in np.arange(par['t'][-1]) for l in np.arange(par['load'][-1])])

    #temp_stor = sum([var['storDchActPower'][s, t] * par['storLimit'][s, t, 3] - var['storChActPower'][s, t] * par['storLimit'][s, t, 2] + var['EminRelaxStor'][s, t] * 200 
    #                 for t in np.arange(par['t'][-1]) for s in np.arange(par['stor'][-1])])

    #temp_v2g = sum([var['v2gDchActPower'][v, t] * par['v2gLimit'][v, t, 6] - var['v2gChActPower'][v, t] * par['v2gLimit'][v, t, 5] + var['EminRelaxEv'][v, t] * 200
    #                for t in np.arange(par['t'][-1]) for v in np.arange(par['v2g'][-1])])

    #temp_rest = sum([var['pImp'][i] * par['buyPrice'][i] - var['pExp'][i] * par['sellPrice'][i] 
    #                 for i in np.arange(par['t'][-1])])

    #obj_value.append(temp_gens + temp_loads + temp_stor + temp_v2g + temp_rest)

    #obj_value = np.reshape(obj_value, (vals.shape[0], 1))

    return sum_gens + sum_loads + sum_stor + sum_v2g + sum_rest


# define population size
pop_size = 50
# define lower and upper bounds for every dimension
#bounds = asarray([(-5.0, 5.0), (-5.0, 5.0)])
bounds = asarray(list(zip(encode(minVals), encode(maxVals))))
# define number of iterations
iters = int(1e5)
# define scale factor for mutation
F = 0.5
# define crossover rate for recombination
cr = 0.7

def differential_evolution(pop_size, bounds, iters, F, cr):
    # initialise population of candidate solutions randomly within the specified bounds
    #pop: cython.float[50] = bounds[:, 0] + (rand(pop_size, len(bounds)) * (bounds[:, 1] - bounds[:, 0]))
    pop = bounds[:, 0] + (rand(pop_size, len(bounds)) * (bounds[:, 1] - bounds[:, 0]))
    # evaluate initial population of candidate solutions
    #obj_all: cython.float[50] = [eso_obj(pop[ind, :]) for ind in range(pop.shape[0])]
    #obj_all = eso_obj(pop)
    obj_all = [eso_obj(pop[ind, :]) for ind in range(pop.shape[0])]
    # find the best performing vector of initial population
    #best_vector: cython.float = pop[argmin(obj_all)]
    best_vector = pop[argmin(obj_all)]
    #best_obj: cython.float = min(obj_all)
    best_obj = min(obj_all)
    #prev_obj: cython.float = best_obj
    prev_obj = best_obj
    # run iterations of the algorithm

    i: cython.int
    j: cython.int
        
    obj_target: cython.float
    obj_trial: cython.float
        
    mutated: cython.float[2424]
    
    for i in tqdm.tqdm(range(iters)):
        # iterate over all candidate solutions
        for j in range(pop_size):
            # choose three candidates, a, b and c, that are not the current one
            candidates = [candidate for candidate in range(pop_size) if candidate != j]
            a, b, c = pop[choice(candidates, 3, replace=False)]
            # perform mutation
            mutated = mutation([a, b, c], F)
            # check that lower and upper bounds are retained after mutation
            mutated = check_bounds(mutated, bounds)
            # Constraints
            #mutated = check_pop(mutated)
            # perform crossover
            trial = crossover(mutated, pop[j], len(bounds), cr)
            # compute objective function value for target vector
            obj_target = eso_obj(pop[j])
            # compute objective function value for trial vector
            obj_trial = eso_obj(trial)
            # perform selection
            if obj_trial < obj_target:
                # replace the target vector with the trial vector
                pop[j] = check_pops(trial)
                # store the new objective function value
                obj_all[j] = eso_obj(pop[j])#obj_trial
        # find the best performing vector at each iteration
        best_obj = min(obj_all)
        # store the lowest objective function value
        if best_obj < prev_obj:
            best_vector = pop[argmin(obj_all)]
            prev_obj = best_obj
            # report progress at each iteration
            # print('Iteration: %d f([%s]) = %.5f' % (i, around(best_vector, decimals=5), best_obj))
    return [best_vector, best_obj]

# perform differential evolution
solution = differential_evolution(pop_size, bounds, iters, F, cr)
#print('\nSolution: f([%s]) = %.5f' % (around(solution[0], decimals=5), solution[1]))

  0%|                                                                                                                          | 0/100000 [00:00<?, ?it/s]<frozen importlib._bootstrap>:219: RuntimeWarning: invalid value encountered in true_divide
<frozen importlib._bootstrap>:219: RuntimeWarning: divide by zero encountered in true_divide
<frozen importlib._bootstrap>:219: RuntimeWarning: invalid value encountered in add
  0%|                                                                                                              | 11/100000 [00:18<47:44:23,  1.72s/it]


KeyboardInterrupt: 